# Reinforcement Learning: Multi-Agent Cooperative-Competitive Learning (Using DRL to learn Tennis)

---

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1. If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01. Thus, the goal of each agent is to keep the ball in play.

<img src="img/tennis.png" alt="drawing" width="500"/>

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Each agent receives its own, local observation. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping.

### Solving the Environment

The task is episodic, and in order to solve the environment, your agents must get an average score of +0.5 (over 100 consecutive episodes, after taking the maximum over both agents). Specifically,

- After each episode, we add up the rewards that each agent received (without discounting), to get a score for each agent. This yields 2 (potentially different) scores. We then take the maximum of these 2 scores.

- This yields a single score for each episode.
The environment is considered solved, when the average (over 100 episodes) of those scores is at least +0.5.

### Academic Sources

The following result is an implementation of a MA-DDPG (Multi Agent Deep Deterministic Policy Agent) inspired by OpenAI's paper titled [Multi-Agent Actor-Critic for Mixed.
Cooperative-Competitive Environments](https://papers.nips.cc/paper/2017/file/68a9750337a418a86fe06c1991a1d64c-Paper.pdf). I have some notes to summarize my takeaways:

- Traditional reinforcement learning approaches such as Q-Learning or policy gradient
are poorly suited to multi-agent environments. One issue is that each agent’s policy is changing
as training progresses, and the environment becomes non-stationary from the perspective of any
individual agent.

- **General Purpose Multi-Agent Learning Algorithm**:
 - (1) leads to learned policies that only use local information (i.e. their own observations) at execution time 
 - (2) does not assume a differentiable model of the environment dynamics or any particular structure on the communication method between agents
 - (3) is applicable not only to cooperative interaction but to competitive or mixed interaction involving both physical and communicative behavior.
 
- **Solution**: actor-critic policy gradient methods where the critic is
augmented with extra information about the policies of other agents, while the actor only has access to local information

- Agents can either be cooperative, competitive, or both and many
algorithms are designed only for a particular nature of interaction. 

- Each agent i aims to maximize its own total expected return.
- Markov Games, Q-Learning, DQN, Policy Gradient Algorithms all struggle for their own reasons

- **Multi-Agent Actor Critic**:
    - Constraints:
        - (1) the learned policies can only use local information (i.e. their own observations) at execution time
        - (2) we do not assume a differentiable model of the environment dynamics
        - (3) we do not assume any particular structure on the communication method between agents 
    - Gradient: 


<img src="img/gradient2.png" alt="drawing" align="center"  width="700"/> -

- Inferring Policies of Other Agents:
    
<img src="img/infering_policies.png" alt="drawing" align="center"  width="700"/> -



---

### Configuration

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Tennis.app")
```

---
### Requirements

Run the next code cell to install a few packages. This line will take a few minutes to run!

In [1]:
#!pip install -r requirements.txt

### Imports 

In [2]:
from unityagents import UnityEnvironment
import numpy as np
import torch
import matplotlib.pyplot as plt
%matplotlib inline

### Unity Environment

In [3]:
env = UnityEnvironment(file_name="Tennis_Windows_x86_64/tennis.exe")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [4]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [5]:
def examine_environment(env, brain):

    # reset the environment
    env_info = env.reset(train_mode=True)[brain_name]

    # number of agents 
    num_agents = len(env_info.agents)
    print('Number of agents:', num_agents)

    # size of each action
    action_size = brain.vector_action_space_size
    print('Size of each action:', action_size)

    # examine the state space 
    states = env_info.vector_observations
    state_size = states.shape[1]
    print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
    print('The state for the first agent looks like:\n', states[0])

In [6]:
examine_environment(env, brain)

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like:
 [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]


### Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agents and receive feedback from the environment.

Once this cell is executed, you will watch the agents' performance, if they select actions at random with each time step.  A window should pop up that allows you to observe the agents.

Of course, as part of the project, you'll have to change the code so that the agents are able to use their experiences to gradually choose better actions when interacting with the environment!

In [7]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])


for i in range(1, 6):                                      # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        test = np.array(states).reshape(1,-1).squeeze()
                
        rewards = env_info.rewards                         # get reward (for each agent)
        print(rewards)
        dones = env_info.local_done                        # see if episode finished
        print(dones)
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -7.38993645 -1.5
 -0.          0.          6.83172083  5.99607611 -0.          0.        ]
[0.0, 0.0]
[False, False]
[0.0, 0.0]
[False, False]
[0.0, 0.0]
[False, False]
[0.0, 0.0]
[False, False]
[0.0, 0.0]
[False, False]
[0.0, 0.0]
[False, False]
[0.0, 0.0]
[False, False]
[0.0, 0.0]
[False, False]
[0.0, 0.0]
[False, False]
[0.0, 0.0]
[False, False]
[0.0, 0.0]
[False, False]
[0.0, 0.0]
[False, False]
[0.0, 0.0]
[False, False]
[0.0, -0.009999999776482582]
[True, True]
Score (max over agents) from episode 1: 0.0
[0.0, 0.0]
[False, False]
[0.0, 0.0]
[False, False]
[0.0, 0.0]
[False, False]
[0.0, 0.0]
[False, False]
[0.0, 0.0]
[False, False]
[0.0, 0.0]
[False, False]
[0.0, 0

When finished, you can close the environment.

In [8]:
env.close()

In [9]:
print([test]*5)


[array([-3.74744868, -0.18940151, -2.34056449, -4.57579613,  6.37251186,
       -1.00070286, -2.34056449, -4.57579613, -0.7474491 , -0.70584118,
       30.        , -5.5567956 ,  6.37251186, -1.92032266, 30.        ,
       -5.5567956 , -0.96895272, -1.32038057,  1.55236363, -6.53779507,
        6.37251186, -2.83994246,  1.55236363, -6.53779507, -8.36013603,
        0.7468465 , 25.39654541,  1.50640261, -6.37251186, -1.00070286,
       25.39654541,  1.50640261, -7.69723701,  0.83862674,  6.62897348,
        0.52540255, -6.37251186, -1.92032266,  6.62897348,  0.52540255,
       -5.72372007,  0.83230692, 19.73517227, -0.45559746, -6.37251186,
       -2.83994246, 19.73517227, -0.45559746]), array([-3.74744868, -0.18940151, -2.34056449, -4.57579613,  6.37251186,
       -1.00070286, -2.34056449, -4.57579613, -0.7474491 , -0.70584118,
       30.        , -5.5567956 ,  6.37251186, -1.92032266, 30.        ,
       -5.5567956 , -0.96895272, -1.32038057,  1.55236363, -6.53779507,
        6.37251

In [111]:
# print(test)
ac = np.array([test] * 5)
actions = np.vstack(ac)
actions = torch.from_numpy(actions).float() # (float)
# print(actions)
actions = actions.detach().numpy().reshape(5,-1).squeeze()
# print(actions)
a = [np.split(action, num_agents)[1] for action in actions]

print(a)
actions = torch.Tensor(a)
print(actions)


[array([-8.360136  ,  0.7468465 , 25.396545  ,  1.5064026 , -6.372512  ,
       -1.0007029 , 25.396545  ,  1.5064026 , -7.697237  ,  0.83862674,
        6.6289735 ,  0.52540255, -6.372512  , -1.9203227 ,  6.6289735 ,
        0.52540255, -5.72372   ,  0.8323069 , 19.735172  , -0.45559746,
       -6.372512  , -2.8399425 , 19.735172  , -0.45559746], dtype=float32), array([-8.360136  ,  0.7468465 , 25.396545  ,  1.5064026 , -6.372512  ,
       -1.0007029 , 25.396545  ,  1.5064026 , -7.697237  ,  0.83862674,
        6.6289735 ,  0.52540255, -6.372512  , -1.9203227 ,  6.6289735 ,
        0.52540255, -5.72372   ,  0.8323069 , 19.735172  , -0.45559746,
       -6.372512  , -2.8399425 , 19.735172  , -0.45559746], dtype=float32), array([-8.360136  ,  0.7468465 , 25.396545  ,  1.5064026 , -6.372512  ,
       -1.0007029 , 25.396545  ,  1.5064026 , -7.697237  ,  0.83862674,
        6.6289735 ,  0.52540255, -6.372512  , -1.9203227 ,  6.6289735 ,
        0.52540255, -5.72372   ,  0.8323069 , 19.735172

In [114]:
print(actions.shape)

torch.Size([5, 24])


### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```